In [ ]:
1. add processing for fixed parameters + remove fixed (and x-axis?) from parameter list
2. pie chart - confirm output values sum to 100?
4. Range - process N -> N_min and N_max in functions
6. add min and max to range?
5. Remove chart specific references in App.js



In [ ]:
# import pandas as pd
# import json
# import re
# from pathlib import Path

# # === Paths
# PARAMETERS_DIR = Path("../01_Inputs/parameters")
# FUNCTIONS_DIR = Path("../01_Inputs/functions")
# OUTPUT_DIR = Path("../../01_App/public/outputs")
# MODELS_CSV = Path("../01_Inputs/models.csv")
# FUNCTIONS_JS = Path("../../01_App/public/functions.js")
# OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# # === Helpers
# def substitute_js_vars(expr: str, range_vars: list[str], param_symbols: list[str], known_locals: set = None) -> str:
#     if known_locals is None:
#         known_locals = set()
#     for var in range_vars:
#         expr = re.sub(fr"\b{var}_min\b", f"params.{var}_min", expr)
#         expr = re.sub(fr"\b{var}_max\b", f"params.{var}_max", expr)

#     def replacer(match):
#         var = match.group(0)
#         return f"params.{var}" if var in param_symbols and var not in known_locals else var

#     return re.sub(r"\b[A-Za-z_]\w*\b", replacer, expr)

# def js_to_latex(expr: str) -> str:
#     expr = expr.strip()
#     expr = expr.replace("Math.min", r"\min").replace("Math.max", r"\max").replace("Math.pow", r"(")
#     expr = re.sub(r"params\.", "", expr)
#     expr = expr.replace("**", "^").replace("*", r" \cdot ")
#     expr = re.sub(r"\b(\w+)\s*/\s*(\w+)\b", r"\\frac{\1}{\2}", expr)
#     expr = re.sub(r"\(([^()]*)\)", r"\\left( \1 \\right)", expr)
#     return expr



# def process_model(code: str, title: str, summary: str) -> tuple[dict, str]:
#     param_path = PARAMETERS_DIR / f"{code}-parameters.csv"
#     func_path = FUNCTIONS_DIR / f"{code}-functions.csv"

#     parameters, range_vars, x_axis_values, x_symbol = load_parameters(param_path)
#     param_symbols = [p["symbol"] for p in parameters]
#     func_df = pd.read_csv(func_path)
#     func_df = func_df.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))

#     outputs = []
#     latex_functions = []
#     compute_lines = [f"function compute_{code}(params) {{", "  const series = [];"]
#     y_value_funcs = []
#     pie_segments = []

#     for _, f in func_df.iterrows():
#         symbol = f["output_symbol"]
#         chart_usage = f["chart_usage"]
#         title_f = f["title"]
    
#         # For multiline y_value, do not replace 'x' with params.x
#         if chart_usage == "y_value":
#             js_expr = substitute_js_vars(f["js_equation"], range_vars, param_symbols, known_locals={"x"})
#             y_value_funcs.append((title_f, js_expr))
#         else:
#             js_expr = substitute_js_vars(f["js_equation"], range_vars, param_symbols)
    
#         outputs.append({
#             "symbol": symbol,
#             "title": title_f,
#             "description": f["description"],
#             "unit": f["unit"],
#             "target": float(f["target"]),
#             "direction": f["direction"],
#             "chart_usage": chart_usage
#         })
    
#         latex_functions.append(f"{symbol} = {js_to_latex(f['js_equation'])}")
    
#         if chart_usage == "segment_value":
#             pie_segments.append((title_f, js_expr))
#         elif chart_usage == "bar_category":
#             compute_lines.append(f"  series.push({{ name: '{title_f}', type: 'bar', data: [ {js_expr} ] }});")
#         elif chart_usage == "bar_stack_component":
#             compute_lines.append(f"  series.push({{ name: '{title_f}', type: 'bar', stack: 'stack', data: [ {js_expr} ] }});")
#         # do NOT have an else here


#     if y_value_funcs:
#         xs_js = str(x_axis_values) if x_axis_values else "Array.from({ length: 21 }, (_, i) => i * 5)"
#         compute_lines.append(f"  const xs = {xs_js};")
#         for title_f, expr in y_value_funcs:
#             key = title_f.replace(' ', '_')
#             compute_lines.append(f"  const y_{key} = xs.map(x => [x, {expr}]);")
#             compute_lines.append(f"  series.push({{ name: '{title_f}', type: 'line', data: y_{key} }});")
#         compute_lines.append(f"  const xAxis = xs;")
#         return_line = f"  return {{ series, xAxis }};"
#     elif pie_segments:
#         pie_data = ",\n    ".join([f"{{ name: '{name}', value: {expr} }}" for name, expr in pie_segments])
#         compute_lines.append("  series.push({")
#         compute_lines.append("    name: 'Energy Mix', type: 'pie', data: [")
#         compute_lines.append(f"      {pie_data}")
#         compute_lines.append("    ]")
#         compute_lines.append("  });")
#         return_line = f"  return {{ series}};"
#     else:
#         return_line = f"  return {{ series}};"

#     compute_lines.append(return_line)
#     compute_lines.append("}")

#     has_line_chart = any(f["chart_usage"] == "y_value" for _, f in func_df.iterrows())
#     all_pie_chart = all(f["chart_usage"] == "segment_value" for _, f in func_df.iterrows())
#     all_bar_chart = all(f["chart_usage"] in ("bar_category", "bar_stack_component") for _, f in func_df.iterrows())

#     xaxis_type = "value" if has_line_chart else "category"   # <--- add this line here!


    
#     if all_pie_chart:
#         option_block = {
#             "tooltip": {"trigger": "item"},
#             "series": []
#         }
        
#     elif all_bar_chart:
#         option_block = {
#             "xAxis": {"type": xaxis_type},
#             "yAxis": {"type": "value"},
#             "tooltip": {"trigger": "item"},
#             "series": []
#         }
#     else:
#         option_block = {
#             "xAxis": {"type": xaxis_type},
#             "yAxis": {"type": "value"},
#             "tooltip": {"trigger": "axis"},
#             "series": []
#         }

    
#     model_json = {
#         "template_id": "model",
#         "color_scheme": "dark",
#         "content": {
#             "functionName": f"compute_{code}",
#             "title": title,
#             "description": summary,
#             "latexFunctions": latex_functions,
#             "option": option_block,
#             "parameters": parameters,
#             "outputs": outputs,
#             "targetType":"any"##update this
#         }
#     }


#     return model_json, "\n".join(compute_lines)

# def process_models():
#     models_df = pd.read_csv(MODELS_CSV)
#     all_compute_code = []

#     for _, row in models_df.iterrows():
#         code = row["code"]
#         title = row["title"]
#         summary = row["summary"]
#         model_json, compute_code = process_model(code, title, summary)

#         with open(OUTPUT_DIR / f"{code}.json", "w") as f:
#             json.dump(model_json, f, indent=2)

#         all_compute_code.append(f"// === {code} ===\n{compute_code}")

#     with open(FUNCTIONS_JS, "w") as f:
#         f.write("\n\n".join(all_compute_code))

# if __name__ == "__main__":
#     process_models()


In [45]:
import pandas as pd
import json
import re
from pathlib import Path

# === Paths
PARAMETERS_DIR = Path("../01_Inputs/parameters")
FUNCTIONS_DIR = Path("../01_Inputs/functions")
OUTPUT_DIR = Path("../../01_App/public/outputs")
MODELS_CSV = Path("../01_Inputs/models.csv")
FUNCTIONS_JS = Path("../../01_App/public/functions.js")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# === Helpers
def substitute_js_vars(expr: str, range_vars: list[str], param_symbols: list[str], known_locals: set = None, fixed_values: dict = None, x_symbol: str = None) -> str:
    if known_locals is None:
        known_locals = set()
    if fixed_values is None:
        fixed_values = {}

    # First, replace range variables _min/_max
    for var in range_vars:
        expr = re.sub(fr"\b{var}_min\b", f"params.{var}_min", expr)
        expr = re.sub(fr"\b{var}_max\b", f"params.{var}_max", expr)

    # Explicit replacement for x_symbol
    if x_symbol:
        expr = re.sub(fr"\b{re.escape(x_symbol)}\b", "x", expr)

    def replacer(match):
        var = match.group(0)
        if var in fixed_values:
            return str(fixed_values[var])
        return f"params.{var}" if var in param_symbols and var not in known_locals else var

    return re.sub(r"\b[A-Za-z_]\w*\b", replacer, expr)


def js_to_latex(expr: str) -> str:
    expr = expr.strip()
    expr = expr.replace("Math.min", r"\min").replace("Math.max", r"\max").replace("Math.pow", r"(")
    expr = re.sub(r"params\.", "", expr)
    expr = expr.replace("**", "^").replace("*", r" \cdot ")
    expr = re.sub(r"\b(\w+)\s*/\s*(\w+)\b", r"\\frac{\1}{\2}", expr)
    expr = re.sub(r"\(([^()]*)\)", r"\\left( \1 \\right)", expr)
    return expr
    
def load_parameters(file_path: Path) -> tuple[list[dict], list[str], list[int], str, dict]:
    df = pd.read_csv(file_path)
    parameters = []
    range_vars = []
    x_axis_values = []
    x_symbol = None
    fixed_values = {}

    for _, row in df.iterrows():
        param_type = row["type"]
        symbol = row["symbol"]

        if param_type == "x-axis":
            x_symbol = symbol
            x_axis_values = list(range(int(row["min"]), int(row["max"]) + 1, int(row["interval"])))
            continue

        if param_type == "fixed":
            fixed_values[symbol] = float(row.get("default", 0))
            continue

        if param_type == "scenario":
            continue  # completely skip

        if param_type not in {"discrete", "continuous", "range"}:
            continue  # ignore unsupported types

        param = {
            "symbol": symbol,
            "title": row["title"],
            "description": row.get("description", row["title"]),
            "unit": row.get("unit", "") if pd.notnull(row.get("unit", "")) else "",
            "type": param_type,
            "min": float(row.get("min", 0)),
            "max": float(row.get("max", 1)),
            "default": float(row.get("default", 0.5)),
            "targetType": "any"
        }

        if param_type == "discrete":
            interval = row.get("interval", "")
            if pd.notnull(interval) and interval != "":
                param["interval"] = float(interval)

        if param_type == "range":
            range_vars.append(symbol)
            if pd.notnull(row.get("rangeMin", None)):
                param["rangeMin"] = float(row["rangeMin"])
            if pd.notnull(row.get("rangeMax", None)):
                param["rangeMax"] = float(row["rangeMax"])

        parameters.append(param)

    return parameters, range_vars, x_axis_values, x_symbol, fixed_values


# Improved scenario handling
def process_scenarios(func_df, scenario_var):
    scenario_info = []
    for _, row in func_df.iterrows():
        scenario_val = str(row["variable"]).strip()
        scenario_title = row["title"].strip()
        js_expr = row["js_equation"].strip()
        scenario_info.append((scenario_var, scenario_val, scenario_title, js_expr))
    return scenario_info



# Process model function (simplified snippet)
def process_model(code: str, title: str, summary: str) -> tuple[dict, str]:
    param_path = PARAMETERS_DIR / f"{code}-parameters.csv"
    func_path = FUNCTIONS_DIR / f"{code}-functions.csv"

    parameters, range_vars, x_axis_values, x_symbol, fixed_values = load_parameters(param_path)
    param_symbols = [p["symbol"] for p in parameters]
    func_df = pd.read_csv(func_path)
    func_df.loc[:, func_df.select_dtypes(include='object').columns] = func_df.select_dtypes(include='object').apply(lambda col: col.str.strip())

    outputs = []
    latex_functions = []
    compute_lines = [f"function compute_{code}(params) {{", "  const series = [];"]
    y_value_funcs = []
    pie_segments = []

    # Scenario handling 
    if "variable" in func_df.columns:
        
        scenario_var = next((p["symbol"] for p in parameters if p["type"] == "scenario"), "R")#None)

        scenario_info = process_scenarios(func_df, scenario_var)
    
        for idx, (var, val, title, js_expr) in enumerate(scenario_info):
            # Replace only the exact variable symbol
            scenario_expr = re.sub(rf"\b{re.escape(var)}\b", val, js_expr)
            final_expr = substitute_js_vars(
                scenario_expr, range_vars, param_symbols, known_locals={"x"}, fixed_values=fixed_values, x_symbol=x_symbol
            )

            output_symbol = func_df["output_symbol"].iloc[idx]
    
            y_value_funcs.append((output_symbol, final_expr))
    
            outputs.append({
                "symbol": output_symbol,
                "title": title,
                "description": title,
                "unit": "",
                "target": None,
                "direction": None,
                "chart_usage": "y_value"
            })
    
            latex_functions.append(f"{output_symbol} = {js_to_latex(scenario_expr)}")

    else:
        # Non-scenario handling
        for _, f in func_df.iterrows():
            symbol = f["output_symbol"]
            chart_usage = f["chart_usage"]
            title_f = f["title"]
            js_expr = f["js_equation"]

            final_expr = substitute_js_vars(
                js_expr, range_vars, param_symbols, known_locals={"x"}, fixed_values=fixed_values, x_symbol=x_symbol
            )


            outputs.append({
                "symbol": symbol,
                "title": title_f,
                "description": f["description"],
                "unit": f["unit"] if pd.notnull(f["unit"]) else "",
                "target": float(f["target"]) if pd.notnull(f["target"]) else None,
                "direction": f["direction"],
                "chart_usage": chart_usage
            })

            latex_functions.append(f"{symbol} = {js_to_latex(js_expr)}")

            if chart_usage == "y_value":
                y_value_funcs.append((symbol, final_expr))
            elif chart_usage == "segment_value":
                pie_segments.append((symbol, final_expr))
            elif chart_usage == "bar_category":
                compute_lines.append(f"  series.push({{ name: '{symbol}', type: 'bar', data: [ {final_expr} ] }});")
            elif chart_usage == "bar_stack_component":
                compute_lines.append(f"  series.push({{ name: '{symbol}', type: 'bar', stack: 'stack', data: [ {final_expr} ] }});")

    # Process y-value functions (for line charts)
    if y_value_funcs:
        xs_js = str(x_axis_values) if x_axis_values else "Array.from({ length: 21 }, (_, i) => i * 5)"
        compute_lines.append(f"  const xs = {xs_js};")
        for key, expr in y_value_funcs:
            compute_lines.append(f"  const y_{key} = xs.map(x => [x, {expr}]);")
            compute_lines.append(f"  series.push({{ name: '{key}', type: 'line', data: y_{key} }});")
        compute_lines.append(f"  const xAxis = xs;")
        return_line = "  return { series, xAxis };"

    elif pie_segments:
        pie_data = ",\n    ".join([f"{{ name: '{name}', value: {expr} }}" for name, expr in pie_segments])
        compute_lines.append("  series.push({")
        compute_lines.append("    name: 'Energy Mix', type: 'pie', data: [")
        compute_lines.append(f"      {pie_data}")
        compute_lines.append("    ]")
        compute_lines.append("  });")
        return_line = "  return { series };"

    else:
        return_line = "  return { series };"

    compute_lines.append(return_line)
    compute_lines.append("}")

    has_line_chart = bool(y_value_funcs)
    all_pie_chart = bool(pie_segments) and not y_value_funcs
    all_bar_chart = all(f["chart_usage"] in ("bar_category", "bar_stack_component") for _, f in func_df.iterrows())

    xaxis_type = "value" if has_line_chart else "category"

    if all_pie_chart:
        option_block = {"tooltip": {"trigger": "item"}, "series": []}
    elif all_bar_chart:
        option_block = {
            "xAxis": {"type": xaxis_type},
            "yAxis": {"type": "value"},
            "tooltip": {"trigger": "item"},
            "series": []
        }
    else:
        option_block = {
            "xAxis": {"type": xaxis_type},
            "yAxis": {"type": "value"},
            "tooltip": {"trigger": "axis"},
            "series": []
        }

    model_json = {
        "template_id": "model",
        "color_scheme": "dark",
        "content": {
            "functionName": f"compute_{code}",
            "title": title,
            "description": summary,
            "latexFunctions": latex_functions,
            "option": option_block,
            "parameters": parameters,
            "outputs": outputs,
            "targetType": "any"
        }
    }

    return model_json, "\n".join(compute_lines)



def process_models():
    models_df = pd.read_csv(MODELS_CSV)
    all_compute_code = []

    for _, row in models_df.iterrows():
        code = row["code"]
        title = row["title"]
        summary = row["summary"]
        model_json, compute_code = process_model(code, title, summary)

        with open(OUTPUT_DIR / f"{code}.json", "w") as f:
            json.dump(model_json, f, indent=2)

        all_compute_code.append(f"// === {code} ===\n{compute_code}")

    with open(FUNCTIONS_JS, "w") as f:
        f.write("\n\n".join(all_compute_code))


if __name__ == "__main__":
    process_models()

